In [1]:
from dataset import download_dataset, generate_taxonomy_dataframe, BeetleSet, TransformsDataset, stratified_split, generate_labels_dict, stratified_split_equal, get_dataloaders
from models import SimpleMultiTaskModel, SequentialMultiTaskModel, SkipMultiTaskModel
from transforms import RandomResizedCrop
from torchvision.transforms import Compose, Resize, RandomVerticalFlip, RandomRotation, Normalize, ToTensor
import torch
from torch.utils.data import Subset, DataLoader
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
import torchvision
import numpy as np
import pandas as pd
from PIL import Image

import json
from pathlib import Path


#from analysis.hierarchy_graph import Graph
import os


import models
import matplotlib.pyplot as plt
import analysis.utility as util
import analysis.cluster as cluster
import fusion_model_dictionaries as fmd
from analysis.metrics import k_fold_stats

In [2]:
pd.set_option('display.max_colwidth', 2000)


In [3]:
models_info = util.load_models_info(util.root)
# Model name, Parent index, index
util.show_models(models_info)[['Model name', 'Parent index', 'Index']]

,Model name,Parent index,Index
0,parallel_0123,0,0
1,sequential_3210,0,1
2,single_0,0,2
3,single_1,0,3
4,single_2,0,4
5,single_3,0,5
6,SH_bce_mean_0123,1,0
7,SH_bce_sum_no_amp_0123,1,1
8,SH_cce_mean_0123,1,2
9,SH_cce_sum_0123,1,3


# K-fold analysis

In [4]:
iters = 1
stds = [0,0,0]
means = [0,0,0]
cis = [(0,0), (0,0), (0,0)]
for seed in [int(np.random.rand() * 100) for i in range(iters)]:
    std1, mean1, ci1 = k_fold_stats(models_info, 3, 2, k = 10, seed = seed, alpha = 0.99)
    std2, mean2, ci2 = k_fold_stats(models_info, 3, 6, k=10, seed = seed, alpha = 0.99)
    std3, mean3, ci3 = k_fold_stats(models_info, 3, 7, k=10, seed = seed, alpha = 0.99)
    stds[0] += std1
    stds[1] += std2
    stds[2] += std3
    means[0] += mean1
    means[1] += mean2
    means[2] += mean3
    cis[0] = (ci1[0] + cis[0][0], ci1[1] + cis[0][1])
    cis[1] = (ci2[0] + cis[1][0], ci2[1] + cis[1][1])
    cis[2] = (ci3[0] + cis[2][0], ci3[1] + cis[2][1])



# decoder bidirectional
print('bidirectional: ', stds[0] / iters, means[0] / iters, (cis[0][0] / iters, cis[0][1] / iters))

# explicit encoder flat fc preprocess
print('flat fc preprocess: ', stds[1] / iters, means[1] / iters, (cis[1][0] / iters, cis[1][1] / iters))

# explicit encoder spatial
print('explicit encoder spatial: ', stds[2] / iters, means[2] / iters, (cis[2][0] / iters, cis[2][1] / iters))


bidirectional:  0.9355072463768117 0.0020909698919139496 (0.9337119192033349, 0.9373025735502886)
flat fc preprocess:  0.941304347826087 0.0020303572567104246 (0.939561063253251, 0.9430476323989228)
explicit encoder spatial:  0.9442028985507246 0.0019661745452441742 (0.9425147218808808, 0.9458910752205687)


In [5]:
# 95%
bidirectional:  (0.9355072463768115, 0.0025575491423942697,
                 (0.9338363449742167, 0.9371781477794062))
flat fc preprocess:  (0.941304347826087, 0.0019540517068456767, (0.9400277241696381, 0.9425809714825358))
explicit encoder spatial:  (0.9442028985507246, 0.001442550150335668, (0.943260449770541, 0.9451453473309082))


SyntaxError: invalid syntax (Temp/ipykernel_21912/279892607.py, line 4)

In [ ]:
# 99%
bidirectional:  (0.9355072463768117, 0.0021438851780505907,              (0.9336664856217894, 0.937348007131834))
flat fc preprocess:  (0.941304347826087, 0.002019313398387199,          (0.9395705456178821, 0.9430381500342918))
explicit encoder spatial:  (0.9442028985507246, 0.003140096618357502,   (0.9415067809222112, 0.946899016179238))
